# Feature Engineering

Now that data is cleaned, we create statistical and temporal features from the sensor readings to improve model performance.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
%matplotlib inline

In [ ]:
# Load cleaned data
train_clean = pd.read_csv('../data/processed/train_ops_cleaned.csv')
val_clean = pd.read_csv('../data/processed/val_ops_cleaned.csv')

# Load target variables
train_tte = pd.read_csv('../data/raw/train_tte.csv')
val_labels = pd.read_csv('../data/raw/validation_labels.csv')

print(f"Training set: {train_clean.shape}")
print(f"Validation set: {val_clean.shape}")
print(f"\nTarget variable (training): {train_tte.shape}")
print(f"Target variable (validation): {val_labels.shape}")

In [ ]:
# Check data structure
print("Training data structure:")
print(train_clean.head())
print(f"\nShape: {train_clean.shape}")
print(f"Unique vehicles: {train_clean['vehicle_id'].nunique()}")
print(f"\nRows per vehicle (sample):")
print(train_clean.groupby('vehicle_id').size().head(10))

## Feature Engineering Functions

In [ ]:
def create_statistical_features(df, sensor_cols):
    """
    Create statistical aggregation features from sensor readings.
    
    Groups time-series data by vehicle_id and computes:
    - Mean: Average sensor value across all time steps
    - Median: Middle value (robust to outliers)
    - Std: Standard deviation (measure of volatility)
    - Min: Minimum sensor reading
    - Max: Maximum sensor reading
    """
    agg_dict = {}
    
    for col in sensor_cols:
        if col in df.columns and col not in ['vehicle_id', 'time_step']:
            agg_dict[f'{col}_mean'] = (col, 'mean')
            agg_dict[f'{col}_median'] = (col, 'median')
            agg_dict[f'{col}_std'] = (col, 'std')
            agg_dict[f'{col}_min'] = (col, 'min')
            agg_dict[f'{col}_max'] = (col, 'max')
    
    # Group by vehicle and aggregate
    features = df.groupby('vehicle_id').agg(**agg_dict).reset_index()
    
    return features


def create_temporal_features(df, sensor_cols):
    """
    Create temporal features from time-series sensor data.
    
    For each vehicle, compute:
    - Last: Most recent sensor reading (final time step)
    - Trend: Linear slope of sensor values over time
    - Volatility: Standard deviation of sensor values
    """
    temporal_features = []
    
    # Focus on cumulative sensors for trend calculation
    cumulative_sensors = [col for col in sensor_cols if '167_' in col]
    
    for vehicle_id, group in df.groupby('vehicle_id'):
        vehicle_features = {'vehicle_id': vehicle_id}
        
        for col in cumulative_sensors:
            if col in group.columns:
                values = group[col].values
                
                # Last value (most recent reading)
                vehicle_features[f'{col}_last'] = values[-1]
                
                # Trend: Linear regression slope (change per time step)
                if len(values) > 1:
                    x = np.arange(len(values))
                    slope = np.polyfit(x, values, 1)[0]
                    vehicle_features[f'{col}_trend'] = slope
                else:
                    vehicle_features[f'{col}_trend'] = 0
                
                # Volatility: Standard deviation
                vehicle_features[f'{col}_volatility'] = np.std(values)
        
        temporal_features.append(vehicle_features)
    
    return pd.DataFrame(temporal_features)

print("Feature engineering functions defined")
print("\nStatistical features: mean, median, std, min, max (aggregated across time steps)")
print("Temporal features: last value, trend (slope), volatility (for cumulative sensors)")

In [ ]:
# Get sensor columns (exclude vehicle_id)
sensor_cols = [col for col in train_clean.columns if col != 'vehicle_id']

print(f"Total sensor columns: {len(sensor_cols)}")
print(f"Sample sensors: {sensor_cols[:5]}")

## Create Training Features

In [ ]:
# Create statistical features for training data
print("Creating statistical features for training data...")
train_statistical = create_statistical_features(train_clean, sensor_cols)

print(f"Statistical features shape: {train_statistical.shape}")
print(f"Sample features: {train_statistical.columns[:10].tolist()}")

In [ ]:
# Create temporal features for training data
print("Creating temporal features for training data...")
train_temporal = create_temporal_features(train_clean, sensor_cols)

print(f"Temporal features shape: {train_temporal.shape}")
print(f"Sample temporal features: {[c for c in train_temporal.columns if 'trend' in c][:5]}")

In [ ]:
# Combine all features for training data
print("Combining all features...")

# Merge statistical and temporal features on vehicle_id
train_features = train_statistical.merge(train_temporal, on='vehicle_id', how='inner')

# Add target variable
train_features = train_features.merge(train_tte[['vehicle_id', 'in_study_repair']], 
                                      on='vehicle_id', how='inner')

print(f"Final training features shape: {train_features.shape}")
print(f"Total engineered features: {train_features.shape[1] - 2}")  # Exclude vehicle_id and target
print(f"\nClass distribution:")
print(train_features['in_study_repair'].value_counts())

In [ ]:
# Save training features
output_path = '../data/features/train_features.csv'
train_features.to_csv(output_path, index=False)

print(f"Training features saved to {output_path}")

## Validation Data Feature Engineering

Apply the same feature engineering transformations to validation data.

In [ ]:
# Create statistical features for validation data
print("Creating statistical features for validation data...")
val_statistical = create_statistical_features(val_clean, sensor_cols)

print(f"Validation statistical features shape: {val_statistical.shape}")

In [ ]:
# Create temporal features for validation data
print("Creating temporal features for validation data...")
val_temporal = create_temporal_features(val_clean, sensor_cols)

print(f"Validation temporal features shape: {val_temporal.shape}")

In [ ]:
# Combine all features for validation data
print("Combining all features...")

# Merge statistical and temporal features
val_features = val_statistical.merge(val_temporal, on='vehicle_id', how='inner')

# Add target variable (validation uses 'class_label' instead of 'in_study_repair')
val_features = val_features.merge(val_labels[['vehicle_id', 'class_label']], 
                                  on='vehicle_id', how='inner')

# Rename to match training data
val_features.rename(columns={'class_label': 'in_study_repair'}, inplace=True)

print(f"Final validation features shape: {val_features.shape}")
print(f"Total engineered features: {val_features.shape[1] - 2}")
print(f"\nClass distribution:")
print(val_features['in_study_repair'].value_counts())

In [ ]:
# Save validation features
output_path = '../data/features/val_features.csv'
val_features.to_csv(output_path, index=False)

print(f"Validation features saved to {output_path}")

## Feature Engineering Summary

We have successfully engineered features from the cleaned sensor data:

**Statistical Features:**
- Mean: Average sensor value across readings
- Median: Middle value, robust to outliers
- Standard Deviation: Measure of sensor value variability
- Min/Max: Range of sensor readings

**Temporal Features:**
- Trend: Rate of change for cumulative sensors (167_X series)
- Last: Most recent sensor reading
- Volatility: Variability measure for cumulative sensors

**Output:**
- Training features: `data/features/train_features.csv`
- Validation features: `data/features/val_features.csv`

These engineered features will be used for model training and evaluation in subsequent notebooks.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
%matplotlib inline

✅ Libraries imported successfully!


# Feature Engineering

Now that data is cleaned, we create statistical and temporal features from the sensor readings to improve model performance.

In [ ]:
# Load cleaned data
train_clean = pd.read_csv('../data/processed/train_ops_cleaned.csv')
val_clean = pd.read_csv('../data/processed/val_ops_cleaned.csv')

# Load target variables
train_tte = pd.read_csv('../data/raw/train_tte.csv')
val_labels = pd.read_csv('../data/raw/validation_labels.csv')

print(f"Training set: {train_clean.shape}")
print(f"Validation set: {val_clean.shape}")
print(f"\nTarget variable (training): {train_tte.shape}")
print(f"Target variable (validation): {val_labels.shape}")

Loading cleaned operational readouts...
✅ Training data loaded: (1122452, 107)
   - Total time series rows: 1,122,452
   - Unique vehicles: 23,550
   - Sensors: 105

✅ Validation data loaded: (196227, 107)
   - Total time series rows: 196,227
   - Unique vehicles: 5,046

🔍 Verification:
   - Training missing values: 0
   - Validation missing values: 0
✅ Training data loaded: (1122452, 107)
   - Total time series rows: 1,122,452
   - Unique vehicles: 23,550
   - Sensors: 105

✅ Validation data loaded: (196227, 107)
   - Total time series rows: 196,227
   - Unique vehicles: 5,046

🔍 Verification:
   - Training missing values: 0
   - Validation missing values: 0


## Data Structure Check

In [ ]:
# Check data structure
print("Training data structure:")
print(train_clean.head())
print(f"\nShape: {train_clean.shape}")
print(f"Unique vehicles: {train_clean['vehicle_id'].nunique()}")
print(f"\nRows per vehicle (sample):")
print(train_clean.groupby('vehicle_id').size().head(10))

📊 Column breakdown:
   - ID columns: 2 ['vehicle_id', 'time_step']
   - Sensor columns: 105

🎯 Features we'll create:
   - 105 sensors × 6 statistics = 630 features

📋 Example - Vehicle 0:
   - Number of readings: 172
   - Time span: 11.2 to 507.4

   Sensor 171_0 values (first 5): [167985.0, 167985.0, 331635.0, 354975.0, 365550.0]


## Feature Engineering Functions

Creating statistical aggregation features from sensor readings.

In [ ]:
def create_statistical_features(df, sensor_cols):
    """
    Create statistical aggregation features from sensor readings.
    
    Groups time-series data by vehicle_id and computes:
    - Mean: Average sensor value across all time steps
    - Median: Middle value (robust to outliers)
    - Std: Standard deviation (measure of volatility)
    - Min: Minimum sensor reading
    - Max: Maximum sensor reading
    """
    agg_dict = {}
    
    for col in sensor_cols:
        if col in df.columns and col not in ['vehicle_id', 'time_step']:
            agg_dict[f'{col}_mean'] = (col, 'mean')
            agg_dict[f'{col}_median'] = (col, 'median')
            agg_dict[f'{col}_std'] = (col, 'std')
            agg_dict[f'{col}_min'] = (col, 'min')
            agg_dict[f'{col}_max'] = (col, 'max')
    
    # Group by vehicle and aggregate
    features = df.groupby('vehicle_id').agg(**agg_dict).reset_index()
    
    return features


def create_temporal_features(df, sensor_cols):
    """
    Create temporal features from time-series sensor data.
    
    For each vehicle, compute:
    - Last: Most recent sensor reading (final time step)
    - Trend: Linear slope of sensor values over time
    - Volatility: Standard deviation of sensor values
    """
    temporal_features = []
    
    # Focus on cumulative sensors for trend calculation
    cumulative_sensors = [col for col in sensor_cols if '167_' in col]
    
    for vehicle_id, group in df.groupby('vehicle_id'):
        vehicle_features = {'vehicle_id': vehicle_id}
        
        for col in cumulative_sensors:
            if col in group.columns:
                values = group[col].values
                
                # Last value (most recent reading)
                vehicle_features[f'{col}_last'] = values[-1]
                
                # Trend: Linear regression slope (change per time step)
                if len(values) > 1:
                    x = np.arange(len(values))
                    slope = np.polyfit(x, values, 1)[0]
                    vehicle_features[f'{col}_trend'] = slope
                else:
                    vehicle_features[f'{col}_trend'] = 0
                
                # Volatility: Standard deviation
                vehicle_features[f'{col}_volatility'] = np.std(values)
        
        temporal_features.append(vehicle_features)
    
    return pd.DataFrame(temporal_features)

print("Feature engineering functions defined")
print("\nStatistical features: mean, median, std, min, max (aggregated across time steps)")
print("Temporal features: last value, trend (slope), volatility (for cumulative sensors)")

 Feature extraction function defined!

 For each vehicle, this creates:
   - 1 vehicle_id column
   - 105 × 6 = 630 feature columns
   - Total: 631 columns


## Get Sensor Columns

In [ ]:
# Get sensor columns (exclude vehicle_id)
sensor_cols = [col for col in train_clean.columns if col != 'vehicle_id']

print(f"Total sensor columns: {len(sensor_cols)}")
print(f"Sample sensors: {sensor_cols[:5]}")

 Testing on Vehicle 0:
   - Number of readings: 172

 Extracted 631 features!

 Example features for sensor 171_0:
   - 171_0_mean:  5,310,002
   - 171_0_max:   10,189,950
   - 171_0_min:   167,985
   - 171_0_std:   2,950,191
   - 171_0_last:  10,189,950
   - 171_0_trend: 59,299.24 (per time step)

 This vehicle's 172 readings → 1 row with 631 features


## Create Training Features

In [ ]:
# Create statistical features for training data
print("Creating statistical features for training data...")
train_statistical = create_statistical_features(train_clean, sensor_cols)

print(f"Statistical features shape: {train_statistical.shape}")
print(f"Sample features: {train_statistical.columns[:10].tolist()}")

 Extracting features for all training vehicles...
Processing 23,550 vehicles...

   Processed 1,000 / 23,550 vehicles (4.2%)
   Processed 1,000 / 23,550 vehicles (4.2%)
   Processed 2,000 / 23,550 vehicles (8.5%)
   Processed 2,000 / 23,550 vehicles (8.5%)
   Processed 3,000 / 23,550 vehicles (12.7%)
   Processed 3,000 / 23,550 vehicles (12.7%)
   Processed 4,000 / 23,550 vehicles (17.0%)
   Processed 4,000 / 23,550 vehicles (17.0%)
   Processed 5,000 / 23,550 vehicles (21.2%)
   Processed 5,000 / 23,550 vehicles (21.2%)
   Processed 6,000 / 23,550 vehicles (25.5%)
   Processed 6,000 / 23,550 vehicles (25.5%)
   Processed 7,000 / 23,550 vehicles (29.7%)
   Processed 7,000 / 23,550 vehicles (29.7%)
   Processed 8,000 / 23,550 vehicles (34.0%)
   Processed 8,000 / 23,550 vehicles (34.0%)
   Processed 9,000 / 23,550 vehicles (38.2%)
   Processed 9,000 / 23,550 vehicles (38.2%)
   Processed 10,000 / 23,550 vehicles (42.5%)
   Processed 10,000 / 23,550 vehicles (42.5%)
   Processed 11,000 /

,vehicle_id,171_0_mean,171_0_max,171_0_min,171_0_std,171_0_last,171_0_trend,666_0_mean,666_0_max,666_0_min,666_0_std,666_0_last,666_0_trend,427_0_mean,427_0_max,427_0_min,427_0_std,427_0_last,427_0_trend,837_0_mean,837_0_max,837_0_min,837_0_std,837_0_last,837_0_trend,...,397_31_trend,397_32_mean,397_32_max,397_32_min,397_32_std,397_32_last,397_32_trend,397_33_mean,397_33_max,397_33_min,397_33_std,397_33_last,397_33_trend,397_34_mean,397_34_max,397_34_min,397_34_std,397_34_last,397_34_trend,397_35_mean,397_35_max,397_35_min,397_35_std,397_35_last,397_35_trend
0,0,5.310002e+06,10189950.0,167985.0,2.950191e+06,10189950.0,59299.241091,197902.168605,372685.0,10787.0,106675.103986,372685.0,2141.220576,9.603583e+07,143970419.0,7413813.0,2.735438e+07,108090562.0,3.635412e+05,27056.273256,41670.0,2296.0,11627.258764,41670.0,222.374078,...,19811.216871,2.273773e+06,4290245.0,261904.0,1.168752e+06,4290245.0,23482.075818,560261.418605,1027186.0,93172.0,268904.723691,1027186.0,5404.486193,74468.220930,138939.0,17874.0,35128.589437,138939.0,704.825745,640.848837,792.0,452.0,97.259305,792.0,1.895385
1,2,3.374853e+06,5648790.0,65520.0,1.582802e+06,5648790.0,161280.982620,192030.212121,289371.0,2226.0,81823.391210,289371.0,8029.846925,1.412227e+08,233978571.0,2891746.0,6.575075e+07,233978571.0,6.686840e+06,40504.787879,68717.0,424.0,17424.536858,68717.0,1713.963235,...,34666.067179,1.042075e+06,1680728.0,10388.0,4.739484e+05,1680728.0,47628.890709,222924.727273,328910.0,2413.0,92176.751548,328910.0,8893.321858,44156.727273,57511.0,1081.0,15670.098558,57511.0,1356.959559,674.484848,793.0,88.0,196.619016,793.0,14.790107
2,3,3.274506e+06,7603590.0,1530.0,2.457206e+06,7603590.0,119071.893863,102097.492958,230831.0,28.0,77016.246497,230831.0,3726.522837,1.175975e+08,273813365.0,111133.0,8.798317e+07,273813365.0,4.231729e+06,45315.323944,100121.0,0.0,33425.292786,100121.0,1613.823608,...,11413.280483,1.146652e+06,2648967.0,221.0,8.722539e+05,2648967.0,42250.217404,228721.971831,531790.0,40.0,176474.445392,531790.0,8544.124413,31454.070423,76915.0,296.0,25077.855652,76915.0,1209.976761,443.070423,712.0,165.0,193.095198,712.0,9.289906


In [ ]:
# Create temporal features for training data
print("Creating temporal features for training data...")
train_temporal = create_temporal_features(train_clean, sensor_cols)

print(f"Temporal features shape: {train_temporal.shape}")
print(f"Sample temporal features: {[c for c in train_temporal.columns if 'trend' in c][:5]}")

In [10]:
# Combine all features for training data
print("Combining all features...")

# Merge statistical and temporal features on vehicle_id
train_features = train_statistical.merge(train_temporal, on='vehicle_id', how='inner')

# Add target variable
train_features = train_features.merge(train_tte[['vehicle_id', 'in_study_repair']], 
                                      on='vehicle_id', how='inner')

print(f"Final training features shape: {train_features.shape}")
print(f"Total engineered features: {train_features.shape[1] - 2}")  # Exclude vehicle_id and target
print(f"\nClass distribution:")
print(train_features['in_study_repair'].value_counts())

 Extracting features for validation vehicles...
Processing 5,046 vehicles...

   Processed 500 / 5,046 vehicles (9.9%)
   Processed 500 / 5,046 vehicles (9.9%)
   Processed 1,000 / 5,046 vehicles (19.8%)
   Processed 1,000 / 5,046 vehicles (19.8%)
   Processed 1,500 / 5,046 vehicles (29.7%)
   Processed 1,500 / 5,046 vehicles (29.7%)
   Processed 2,000 / 5,046 vehicles (39.6%)
   Processed 2,000 / 5,046 vehicles (39.6%)
   Processed 2,500 / 5,046 vehicles (49.5%)
   Processed 2,500 / 5,046 vehicles (49.5%)
   Processed 3,000 / 5,046 vehicles (59.5%)
   Processed 3,000 / 5,046 vehicles (59.5%)
   Processed 3,500 / 5,046 vehicles (69.4%)
   Processed 3,500 / 5,046 vehicles (69.4%)
   Processed 4,000 / 5,046 vehicles (79.3%)
   Processed 4,000 / 5,046 vehicles (79.3%)
   Processed 4,500 / 5,046 vehicles (89.2%)
   Processed 4,500 / 5,046 vehicles (89.2%)
   Processed 5,000 / 5,046 vehicles (99.1%)
   Processed 5,000 / 5,046 vehicles (99.1%)

 Validation feature extraction complete!

 Vali

In [ ]:
# Save training features
output_path = '../data/features/train_features.csv'
train_features.to_csv(output_path, index=False)

print(f"Training features saved to {output_path}")

## Validation Data Feature Engineering

Apply the same feature engineering transformations to validation data.

In [14]:
# Create statistical features for validation data
print("Creating statistical features for validation data...")
val_statistical = create_statistical_features(val_clean, sensor_cols)

print(f"Validation statistical features shape: {val_statistical.shape}")

Training set shape: (23550, 632)
Training label distribution:
in_study_repair
0    21278
1     2272
Name: count, dtype: int64

Validation set shape: (5046, 632)
Validation label distribution:
in_study_repair
0    4910
1     136
Name: count, dtype: int64


In [ ]:
# Create temporal features for validation data
print("Creating temporal features for validation data...")
val_temporal = create_temporal_features(val_clean, sensor_cols)

print(f"Validation temporal features shape: {val_temporal.shape}")

✓ Saved train_features.csv
✓ Saved val_features.csv

Feature engineering complete! Ready for model training.


In [6]:
# Combine all features for validation data
print("Combining all features...")

# Merge statistical and temporal features
val_features = val_statistical.merge(val_temporal, on='vehicle_id', how='inner')

# Add target variable (validation uses 'class_label' instead of 'in_study_repair')
val_features = val_features.merge(val_labels[['vehicle_id', 'class_label']], 
                                  on='vehicle_id', how='inner')

# Rename to match training data
val_features.rename(columns={'class_label': 'in_study_repair'}, inplace=True)

print(f"Final validation features shape: {val_features.shape}")
print(f"Total engineered features: {val_features.shape[1] - 2}")
print(f"\nClass distribution:")
print(val_features['in_study_repair'].value_counts())

Extracting trend features...


NameError: name 'feature_engine' is not defined

In [ ]:
# Save validation features
output_path = '../data/features/val_features.csv'
val_features.to_csv(output_path, index=False)

print(f"Validation features saved to {output_path}")

## Feature Engineering Summary

We have successfully engineered features from the cleaned sensor data:

**Statistical Features:**
- Mean: Average sensor value across readings
- Median: Middle value, robust to outliers
- Standard Deviation: Measure of sensor value variability
- Min/Max: Range of sensor readings

**Temporal Features:**
- Trend: Rate of change for cumulative sensors (167_X series)
- Last: Most recent sensor reading
- Volatility: Variability measure for cumulative sensors

**Output:**
- Training features: `data/features/train_features.csv`
- Validation features: `data/features/val_features.csv`

These engineered features will be used for model training and evaluation in subsequent notebooks.